In [1]:
!nvidia-smi

Sat Jan  8 14:18:21 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.86       Driver Version: 470.86       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
| 26%   43C    P5    43W / 250W |   3265MiB / 10985MiB |     23%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import matplotlib
%matplotlib inline
import matplotlib.pylab as plt

import IPython.display as ipd

import sys
sys.path.append('waveglow/')
import numpy as np
import torch

from hparams import create_hparams
from model import Tacotron2
from layers import TacotronSTFT, STFT
from audio_processing import griffin_lim
from train import load_model
from text import text_to_sequence
from denoiser import Denoiser

In [3]:
import numpy as np
from scipy.io.wavfile import write

In [4]:
# !conda install -n tts_base ipykernel --update-deps --force-reinstall
# !conda install -c conda-forge librosa -y
# !conda install -c conda-forge inflect -y

In [5]:
import torch
# waveglow = torch.hub.load('nvidia/DeepLearningExamples', 'nvidia_waveglow')
waveglow_path = 'waveglow_256channels_universal_v5.pt'
waveglow = torch.load(waveglow_path)['model']

In [6]:
waveglow = waveglow.remove_weightnorm(waveglow)
waveglow = waveglow.to('cuda')
waveglow.eval()

WaveGlow(
  (upsample): ConvTranspose1d(80, 80, kernel_size=(1024,), stride=(256,))
  (WN): ModuleList(
    (0): WN(
      (in_layers): ModuleList(
        (0): Conv1d(256, 512, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): Conv1d(256, 512, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
        (2): Conv1d(256, 512, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(4,))
        (3): Conv1d(256, 512, kernel_size=(3,), stride=(1,), padding=(8,), dilation=(8,))
        (4): Conv1d(256, 512, kernel_size=(3,), stride=(1,), padding=(16,), dilation=(16,))
        (5): Conv1d(256, 512, kernel_size=(3,), stride=(1,), padding=(32,), dilation=(32,))
        (6): Conv1d(256, 512, kernel_size=(3,), stride=(1,), padding=(64,), dilation=(64,))
        (7): Conv1d(256, 512, kernel_size=(3,), stride=(1,), padding=(128,), dilation=(128,))
      )
      (res_skip_layers): ModuleList(
        (0): Conv1d(256, 512, kernel_size=(1,), stride=(1,))
        (1): Conv1d(256, 512, k

In [8]:
from IPython.display import Audio
from scipy.io.wavfile import write

# load tacotron model

# tacotron2 = torch.hub.load('nvidia/DeepLearningExamples', 'nvidia_tacotron2')
# tacotron2 = tacotron2.to('cuda')
# tacotron2.eval()

hparams = create_hparams()
hparams.sampling_rate = 22050

# checkpoint_list = [2400, 16000, 19200, 25600, 29600, 39200, 41600, 42400, 53600, 62400]
checkpoint_list = [39200, 42400, 53600, 62400]
for c in checkpoint_list:
    checkpoint_path = f"./outdir_200/checkpoint_{c}"

    tacotron2 = load_model(hparams)
    tacotron2.load_state_dict(torch.load(checkpoint_path)['state_dict'])
    _ = tacotron2.eval()
    # _ = tacotron2.cuda().eval()
    # del tacotron2

    text = "안녕하세요. 오늘은 금요일 입니다."

    # utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_tts_utils')
    # sequences, lengths = utils.prepare_input_sequence([text])

    sequence = np.array(text_to_sequence(text, ['english_cleaners']))[None, :]
    sequence = torch.from_numpy(sequence).to(device='cuda', dtype=torch.int64).cpu()
    # device = torch.device("cpu")

    # run the models
    with torch.no_grad():
        _, mel, _, _ = tacotron2.cpu().inference(sequence.cpu())
        mel = mel.to('cuda')
        audio = waveglow.infer(mel)
    audio_numpy = audio[0].data.cpu().numpy()
    rate = 22050

    Audio(data=audio_numpy, rate=rate)

    write(f"./output_wav/test200_kor_{c}.wav", rate, audio_numpy)
    
    # del waveglow
    del tacotron2
    del mel
    torch.cuda.empty_cache()

In [12]:
# Audio(filename='test_8000.wav')
# Audio("http://www.nch.com.au/acm/8k16bitpcm.wav") 

In [13]:
audio_numpy.shape

(256000,)

In [14]:
sum(audio_numpy)

-1.1371152866595553

In [13]:
del waveglow

In [14]:
!nvidia-smi

Sat Jan  8 12:58:22 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.86       Driver Version: 470.86       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
| 33%   47C    P5    42W / 250W |   3681MiB / 10985MiB |     24%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:
!nvidia-smi

Sat Jan  8 12:57:59 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.86       Driver Version: 470.86       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
| 33%   51C    P2    73W / 250W |   3632MiB / 10985MiB |      5%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
import IPython
IPython.__version__

'7.31.0'

In [2]:
import gc, torch
gc.collect()
torch.cuda.empty_cache()

In [1]:
!nvidia-smi

Thu Jan  6 12:34:15 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.86       Driver Version: 470.86       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
| 27%   47C    P3    55W / 250W |   8107MiB / 10985MiB |     23%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------